In [1]:
#QUESTION 1
import numpy as np
import pandas as pd

In [24]:
df = pd.read_excel('data.xlsx')
df.head()

,Age,SiteSpending,SiteTime,RecommendImpression,Education,WorkType,Gender,Region,Salary
0,32,314.06,30.14,0,Degree,Private sector,Male,London,60173.49
1,20,3758.36,149.36,4,GCSE,Private sector,Female,South East,42965.45
2,36,601.72,21.87,0,Masters,Private sector,Male,East of England,54924.41
3,21,44.89,182.80,9,Masters,Private sector,Female,Northern Ireland,26734.99
4,24,614.80,194.34,0,GCSE,Private sector,Male,Scotland,15325.23


In [26]:
#Add a salaries column to the data and if salary is greater than 35000 then assign 1 else 0
label = []
for row in df['Salary']:
        if row >= 35000: label.append(1)
        elif row < 35000: label.append(0)
# data['salaries'] = np.where(data['Salary'] > 35000, 1, 0)
df.drop(['Education', 'WorkType', 'Gender', 'Region'], axis=1)
df['label'] = label

In [27]:
df.head()

,Age,SiteSpending,SiteTime,RecommendImpression,Education,WorkType,Gender,Region,Salary,label
0,32,314.06,30.14,0,Degree,Private sector,Male,London,60173.49,1
1,20,3758.36,149.36,4,GCSE,Private sector,Female,South East,42965.45,1
2,36,601.72,21.87,0,Masters,Private sector,Male,East of England,54924.41,1
3,21,44.89,182.80,9,Masters,Private sector,Female,Northern Ireland,26734.99,0
4,24,614.80,194.34,0,GCSE,Private sector,Male,Scotland,15325.23,0


In [28]:
# convert data to numeric

df['Age'] = pd.to_numeric(df['Age'], errors='coerce')
df['Salary'] = pd.to_numeric(df['Salary'], errors='coerce')
df['label'] = pd.to_numeric(df['label'], errors='coerce')
df['SiteSpending'] = pd.to_numeric(df['SiteSpending'], errors='coerce')
df['SiteTime'] = pd.to_numeric(df['SiteTime'], errors='coerce')

df.dtypes



Age                      int64
SiteSpending           float64
SiteTime               float64
RecommendImpression      int64
Education               object
WorkType                object
Gender                  object
Region                  object
Salary                 float64
label                    int64
dtype: object

In [31]:
from sklearn.model_selection import train_test_split

# split data into train and test sets

seed = 7

test_size = 0.33

X = df.drop(['label'], axis=1)

Y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)


In [32]:
# convert X_train to numpy array

X_train = X_train.to_numpy()

#convert y_train to numpy array

y_train = y_train.to_numpy()

#convert X_test to numpy array

X_test = X_test.to_numpy()

#convert y_test to numpy array

y_test = y_test.to_numpy()

In [33]:
class KNearestNeighbors():
    def __init__(self, X_train, y_train, n_neighbors=5, weights='uniform'):

        self.X_train = X_train
        self.y_train = y_train

        self.n_neighbors = n_neighbors
        self.weights = weights

        self.n_classes = 3

    def euclidian_distance(self, a, b):
        return np.sqrt(np.sum((a - b)**2, axis=1))

    def kneighbors(self, X_test, return_distance=False):

        dist = []
        neigh_ind = []

        point_dist = [self.euclidian_distance(x_test, self.X_train) for x_test in X_test]

        for row in point_dist:
            enum_neigh = enumerate(row)
            sorted_neigh = sorted(enum_neigh,
                                  key=lambda x: x[1])[:self.n_neighbors]

            ind_list = [tup[0] for tup in sorted_neigh]
            dist_list = [tup[1] for tup in sorted_neigh]

            dist.append(dist_list)
            neigh_ind.append(ind_list)

        if return_distance:
            return np.array(dist), np.array(neigh_ind)

        return np.array(neigh_ind)

    def predict(self, X_test):

        if self.weights == 'uniform':
            neighbors = self.kneighbors(X_test)
            y_pred = np.array([
                np.argmax(np.bincount(self.y_train[neighbor]))
                for neighbor in neighbors
            ])

            return y_pred

        if self.weights == 'distance':

            dist, neigh_ind = self.kneighbors(X_test, return_distance=True)

            inv_dist = 1 / dist

            mean_inv_dist = inv_dist / np.sum(inv_dist, axis=1)[:, np.newaxis]

            proba = []

            for i, row in enumerate(mean_inv_dist):

                row_pred = self.y_train[neigh_ind[i]]

                for k in range(self.n_classes):
                    indices = np.where(row_pred == k)
                    prob_ind = np.sum(row[indices])
                    proba.append(np.array(prob_ind))

            predict_proba = np.array(proba).reshape(X_test.shape[0],
                                                    self.n_classes)

            y_pred = np.array([np.argmax(item) for item in predict_proba])

            return y_pred

    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)

        return float(sum(y_pred == y_test)) / float(len(y_test))




In [35]:
# Run the function above on the data

knn = KNearestNeighbors(X_train, y_train, n_neighbors=3, weights='uniform')
knn.score(X_test, y_test)

TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [133]:
# load test.xlsx

test = pd.read_excel('test.xlsx')

In [134]:
test.head()

,Age,SiteSpending,SiteTime,RecommendImpression,Education,WorkType,Gender,Region,Salary
0,37,42.43,103.79,8,Degree,Unemployed,Female,London,NaN
1,35,861.42,155.85,6,Degree,Private sector,Female,South East,NaN
2,37,243.32,195.73,7,Degree,Private sector,Female,South East,NaN
3,51,2584.31,38.28,2,Degree,Private sector,Female,London,NaN
4,25,440.17,18.13,2,GCSE,Unemployed,Female,West Midlands,NaN


In [135]:
test.drop(['Education', 'WorkType', 'Gender', 'Region'], axis=1, inplace=True)

In [136]:
test.head()

,Age,SiteSpending,SiteTime,RecommendImpression,Salary
0,37,42.43,103.79,8,NaN
1,35,861.42,155.85,6,NaN
2,37,243.32,195.73,7,NaN
3,51,2584.31,38.28,2,NaN
4,25,440.17,18.13,2,NaN


In [137]:
# Convert to numpy

test = test.to_numpy()

In [138]:
# test the model

knn.predict(test)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [ ]:
#QUESTION 2

import pandas as pd
from sklearn.preprocessing import LabelEncoder

df=pd.read_csv(r"Desktop/a7py/Dataaa.csv")
df


